In [70]:
import os
import sounddevice as sd
import numpy as np
import time
from time import time
from time import sleep
from scipy.io.wavfile import write
import argparse as ap
import tensorflow as tf
import tensorflow_io as tfio
import uuid
import redis
import psutil
# import myConnection as mc
from datetime import datetime
import argparse as ap
import pandas as pd
import random

In [71]:
try:
    os.chdir('./datasets/dsl_data/')
except:
    print("")

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [72]:
parser = ap.ArgumentParser()

parser.add_argument('--resolution', default=8000, type=int, help="Resolution for capturing audio")
# blocksize
#parser.add_argument('--blocksize', default=32000, type=int, help="Blocksize for captured audio, change only if you previously changed")
parser.add_argument('--downsampling_rate', default=8000, type=int, help="Resolution for capturing audio")
parser.add_argument('--device', default=0, type=int, help="Default device is 0, change for others")


parser.add_argument('--output_directory', default='./AudioFiles',type=str, help='Used to specify output folder')


args = parser.parse_args(['--device','14','--resolution','8000' ])
#args = parser.parse_args()

In [73]:
blocksize = 4 * args.resolution
LABELS = ['change languagenone', 'activatemusic', 'deactivatelights', 'increasevolume', 'decreasevolume', 'increaseheat', 'decreaseheat', 'nannan']

In [74]:
print(LABELS)

['change languagenone', 'activatemusic', 'deactivatelights', 'increasevolume', 'decreasevolume', 'increaseheat', 'decreaseheat', 'nannan']


# Necessary preprocessing args

In [75]:
frame_length_in_s = 0.04#0.032*2 # /2 for resnet18
frame_step_in_s  = frame_length_in_s#frame_length_in_s

PREPROCESSING_ARGS = {
    'downsampling_rate': args.resolution,
    'frame_length_in_s': frame_length_in_s,
    'frame_step_in_s': frame_step_in_s,
}

num_mel_bins = (int) ((args.resolution - args.resolution * PREPROCESSING_ARGS['frame_length_in_s'])/(args.resolution*PREPROCESSING_ARGS['frame_step_in_s']))+1
# print(num_mel_bins)

PREPROCESSING_ARGS = {
    **PREPROCESSING_ARGS,
    'num_mel_bins': num_mel_bins,
    'lower_frequency': 20,   #40
    'upper_frequency': args.resolution/2, #4000
}

downsampling_rate = PREPROCESSING_ARGS['downsampling_rate']
sampling_rate_int64 = tf.cast(downsampling_rate, tf.int64)
frame_length = int(downsampling_rate * PREPROCESSING_ARGS['frame_length_in_s'])
#print("Frame_length: {}".format(frame_length))
frame_step = int(downsampling_rate * PREPROCESSING_ARGS['frame_step_in_s'])
#print("Frame_length: {}".format(frame_step))
num_spectrogram_bins = frame_length // 2 + 1
num_mel_bins = PREPROCESSING_ARGS['num_mel_bins']
lower_frequency = PREPROCESSING_ARGS['lower_frequency']
upper_frequency = PREPROCESSING_ARGS['upper_frequency']

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
    num_mel_bins=num_mel_bins,
    num_spectrogram_bins=num_spectrogram_bins,
    sample_rate=downsampling_rate,
    lower_edge_hertz=lower_frequency,
    upper_edge_hertz=upper_frequency
)

In [76]:
modelName = "model_24"

interpreter = tf.lite.Interpreter(model_path=f'./tflite_models/{modelName}.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [77]:
def get_audio_from_numpy(indata):
    indata = tf.convert_to_tensor(indata, dtype=tf.float32)
    #print("Shape of indata: ",tf.reduce_max(indata))
    indata = 2 * ((indata + 32768) / (32767 + 32768)) -1
    indata = tf.squeeze(indata)
    #print("After of indata: ",tf.reduce_max(indata))
    return indata

def get_spectrogram(indata, frame_length_in_s, frame_step_in_s):
    data = get_audio_from_numpy(indata)
    
    sampling_rate_float32 = tf.cast(args.downsampling_rate, tf.float32)
    frame_length = int(frame_length_in_s * sampling_rate_float32)
    frame_step = int(frame_step_in_s * sampling_rate_float32)

    stft = tf.signal.stft(
        data,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    spectrogram = tf.abs(stft)

    return spectrogram

In [78]:
state = False

In [79]:
def send_prediction_as_mqtt(predicted_label):
    #print(type(predicted_label))
    #print(predicted_label.shape)
    #print(predicted_label)
    print("MAX: ")
    print(predicted_label.max())
    index = ( np.where(predicted_label == predicted_label.max() )  )
    index = index[0][0]
    print(index)
    print(LABELS[index])
    print()

print(LABELS[index])

In [80]:
def prediction_on_indata(indata):
    frame_length_in_s = 0.04
    frame_step_in_s   = frame_length_in_s
    global state
    audio = get_audio_from_numpy(indata)
    
    frame_length = int(frame_length_in_s * args.resolution)
    frame_step = int(frame_step_in_s * args.resolution)
    stft = tf.signal.stft(
        audio,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    
    spectrogram = tf.abs(stft)
    
    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, 0)  # batch axis
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)  # channel axis
    mfcss = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
    #print("Shape ",input_details[0])
    interpreter.set_tensor(input_details[0]['index'], mfcss)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    print("change languagenone",output[0][0]*100,"%")
    print("activatemusic",output[0][1]*100,"%")
    print("deactivatelights",output[0][2]*100,"%")
    print("increasevolume",output[0][3]*100,"%")
    print("decreasevolume",output[0][4]*100,"%")
    print("increaseheat",output[0][5]*100,"%")
    print("decreaseheat",output[0][6]*100,"%")
    #print("nannan",output[0][7]*100,"%")
    
    send_prediction_as_mqtt(output[0])
    
    return state

In [81]:
values = sd.query_devices()
device = 0

for value in values:
    if value['name'] == 'default':
        device = value['index']

In [82]:
def callback(indata, frames, callback_time, status):
    timestamp = time()
    global state
    global mac_address
    #if is_silence(indata) == 0 :
        #calculate next step of FSM!
    prediction_on_indata(indata)
    print("Elapsed time: ",time()-timestamp)

In [83]:
print(LABELS)

['change languagenone', 'activatemusic', 'deactivatelights', 'increasevolume', 'decreasevolume', 'increaseheat', 'decreaseheat', 'nannan']


def main():

    #print(LABELS)
    while True:
        with sd.InputStream(device=args.device, channels=1, dtype='int16', samplerate=args.resolution, blocksize=blocksize, callback=callback):
            print("") # to print a new line, improving readability in the terminal

if __name__ == '__main__':
    main()

# Test

In [93]:
print("Test area")
def callback(indata, frames, callback_time, status):
    """This is called (from a separate thread) for each audio block."""
    timestamp = time()
    # print(is_silence(indata))
    # print(type(indata))  # Type is numpy.ndarray
    
    print("Noise!")
    write(f'./{args.output_directory}/{timestamp}.wav', args.resolution, indata)
    filesize_in_bytes = os.path.getsize(f'./{args.output_directory}/{timestamp}.wav')
    filesize_in_kb = filesize_in_bytes / 1024
    print(f'Size: {filesize_in_kb:.2f}KB')

# 10 fron on screen microphone
# 14 from microphone nada?

def test():
    with sd.InputStream(device=args.device, channels=1, dtype='int16', samplerate=args.resolution, blocksize=blocksize, callback=callback):
        while True:
            key = input()
            if key in ('q', 'Q'):
                print('Stop recording.')
                break
                
                
filename1 = "./Train_Dataset_Truncated/0_change languagenone.wav"
filename2 = "./Train_Dataset_Truncated/15_deactivatelights.wav"
    
    
filename21 = "./Train_Dataset_Truncated/113_increasevolume.wav"
filename22 = "./Train_Dataset_Truncated/151_increasevolume.wav"
filename23 = "./Train_Dataset_Truncated/212_increaseheat.wav"
    
filename3 = "./AudioFiles/1678717382.0476763.wav"   
filename4 = "./AudioFiles/1678717386.5524414.wav"
filename5 = "./AudioFiles/1678717391.0566754.wav"
filename6 = "./AudioFiles/1678717395.5633824.wav"  
filename7 = "./AudioFiles/1678717400.0677488.wav" 
                
def test2(filename):
    print("Prediction for",filename)
    frame_length_in_s = 0.04
    frame_step_in_s   = frame_length_in_s
    global state
    audio_binary = tf.io.read_file(filename)
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)
    audio = tf.squeeze(audio, axis=-1) #all our audio are mono, drop extra axis
    
    frame_length = int(frame_length_in_s * args.resolution)
    frame_step = int(frame_step_in_s * args.resolution)
    stft = tf.signal.stft(
        audio,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=frame_length
    )
    
    spectrogram = tf.abs(stft)
    
    mel_spectrogram = tf.matmul(spectrogram, linear_to_mel_weight_matrix)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, 0)  # batch axis
    log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)  # channel axis
    mfcss = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
    #print("Shape ",input_details[0])
    interpreter.set_tensor(input_details[0]['index'], mfcss)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    print("change languagenone",output[0][0]*100,"%")
    print("activatemusic",output[0][1]*100,"%")
    print("deactivatelights",output[0][2]*100,"%")
    print("increasevolume",output[0][3]*100,"%")
    print("decreasevolume",output[0][4]*100,"%")
    print("increaseheat",output[0][5]*100,"%")
    print("decreaseheat",output[0][6]*100,"%")
    #print("nannan",output[0][7]*100,"%")
    
    send_prediction_as_mqtt(output[0])


#test()

test2(filename1)
test2(filename2)
test2(filename21)
test2(filename22)
test2(filename23)
test2(filename3)
test2(filename4)
test2(filename5)
test2(filename6)
test2(filename7)

Test area
Prediction for ./Train_Dataset_Truncated/0_change languagenone.wav
change languagenone 99.99960660934448 %
activatemusic 9.462366445944781e-08 %
deactivatelights 1.3945601617937342e-08 %
increasevolume 0.0003364256144777755 %
decreasevolume 1.0589629173551884e-07 %
increaseheat 1.4364825773327539e-06 %
decreaseheat 5.9736004232036066e-05 %
MAX: 
0.99999607
0
change languagenone

Prediction for ./Train_Dataset_Truncated/15_deactivatelights.wav
change languagenone 0.22138759959489107 %
activatemusic 0.004535994230536744 %
deactivatelights 37.21541464328766 %
increasevolume 20.413024723529816 %
decreasevolume 39.94375467300415 %
increaseheat 0.19499043701216578 %
decreaseheat 2.0068882033228874 %
MAX: 
0.39943755
4
decreasevolume

Prediction for ./Train_Dataset_Truncated/113_increasevolume.wav
change languagenone 7.916805405683291e-09 %
activatemusic 4.833198945231043e-10 %
deactivatelights 2.5576694095974517e-07 %
increasevolume 99.99988079071045 %
decreasevolume 0.000104019204